In [ ]:
import pandas as pd 
import ast


In [ ]:
# Crea una lista vacía llamada data_list para almacenar los datos del archivo JSON.
data_list = []

# Especifica la ruta del archivo JSON que se va a leer.
file_path = 'australian_users_items.json'

# Abre el archivo en modo de lectura ('r') con codificación UTF-8.
with open(file_path, 'r', encoding='utf-8') as file:
    # Itera sobre cada línea en el archivo.
    for line in file:
        try:
            # Intenta evaluar la línea como un diccionario de Python utilizando ast.literal_eval.
            # Agrega el diccionario resultante a la lista data_list.
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            # Si ocurre un error al evaluar la línea, imprime un mensaje de error y continúa con la siguiente línea.
            print(f"Error en la línea: {line}")
            continue

# Crea un DataFrame de pandas a partir de la lista de diccionarios almacenada en data_list.
data_items = pd.DataFrame(data_list)

In [ ]:

# Utiliza el método 'explode' para descomponer la columna 'items' que contiene listas en varias filas duplicando las demás columnas.
data_items1 = data_items.explode(['items'])

# Utiliza 'pd.json_normalize' para aplanar las columnas JSON anidadas dentro de la columna 'items'.
# Luego, establece el índice de 'data_items2' para que coincida con el índice original de 'data_items1'.
data_items2 = pd.json_normalize(data_items1['items']).set_index(data_items1['items'].index)

# Combina los DataFrames 'data_items1' y 'data_items2' a lo largo de las columnas.
data_items3 = pd.concat([data_items2, data_items1], axis=1)

# Elimina columnas innecesarias: 'items_count', 'user_url', 'item_name', 'steam_id'.
data_items3.drop(['items_count', 'user_url', 'item_name', 'steam_id'], axis=1, inplace=True)

# Elimina filas con valores nulos en columnas específicas: 'item_id' y 'playtime_forever'.
data_items3 = data_items3.dropna(subset=['item_id', 'playtime_forever'])

# Elimina columnas 'items' y 'playtime_2weeks'.
data_items3.drop(['items', 'playtime_2weeks'], axis=1, inplace=True)

# Filtra las filas donde 'playtime_forever' no sea igual a 0.
data_items3 = data_items3[data_items3['playtime_forever'] != 0]

# Selecciona columnas específicas.
data_items3 = data_items3[['user_id', 'item_id', 'playtime_forever']]

# Elimina duplicados en el DataFrame.
data_items3 = data_items3.drop_duplicates()

# Elimina filas con valores nulos en todo el DataFrame.
data_items3 = data_items3.dropna()

In [ ]:
data_items3

In [ ]:
data_items3.to_csv('user_items.csv', index=False)